In [1]:
from utils.config_loader import load_config
from processing.preprocessing import preprocess_data
from processing.normalize import normalize_data
from processing.labelizer import get_labels

config = load_config()

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 96 from PyObject


In [2]:
preprocess_data(config)

In [ ]:
normalize_data(config)

In [2]:
get_labels(config)